# Chat 对象使用示例

这个示例展示了如何使用 Chat 对象进行对话管理，包括：
- 从 .env 文件加载配置
- 基本对话功能
- 异步对话
- 对话历史管理
- 错误处理
- 高级功能

## 1. 环境准备和导入模块

In [1]:
import os
from datetime import datetime

# 导入 chattool 相关模块
from chattool import Chat, debug_log

In [2]:
debug_log()

Current version: 4.1.0

Check the value OPENAI_API_BASE_URL:
None

Check the value OPENAI_API_BASE: This will override OPENAI_API_BASE_URL if both are set.
https://4.0.wokaai.com/v1

Your default OPENAI_API_MODEL:
claude-3-5-sonnet-20240620

Please verify your API key:
sk-n3Qws***************************************7e8f2073

The model list:
Model list functionality temporarily disabled

Test response: hello world! 你好！
Response functionality temporarily disabled

Debug is finished.


True

## 2. 创建 Chat 对象

使用环境变量自动配置

In [3]:
# 方法1: 使用默认配置（自动从环境变量读取）
chat = Chat()
chat

<Chat with 0 messages, model: claude-3-5-sonnet-20240620>

In [4]:
# 方法2: 使用显式配置
chat = Chat(
    api_key=os.getenv('OPENAI_API_KEY'),
    api_base=os.getenv('OPENAI_API_BASE'),
    model=os.getenv('OPENAI_API_MODEL', 'gpt-3.5-turbo'),
    temperature=0.7,
    max_tokens=1000
)
print("✅ Chat 对象创建成功（使用显式配置）")
chat.user('hello')
chat.get_response()

✅ Chat 对象创建成功（使用显式配置）


<ChatResponse: stop, tokens: 18>

## 3. 基本对话示例

In [7]:
chat = Chat(model='gpt-3.5-turbo')
# 设置系统提示
chat.system("你是一个友善的AI助手，请用简洁明了的方式回答问题。")

# 进行第一轮对话
print("=== 第一轮对话 ===")
response1 = chat.user("你好！请简单介绍一下自己。").get_response()
print(f"用户: 你好！请简单介绍一下自己。")
print(f"助手: {response1.content}")
print(f"Token使用: {response1.total_tokens}")

=== 第一轮对话 ===
用户: 你好！请简单介绍一下自己。
助手: 你好！我是一个由人工智能驱动的助手，能够帮助回答问题、提供信息、协助写作和解决各种任务。如果你有任何问题，随时告诉我！
Token使用: 78


In [8]:
# 继续对话
print("=== 第二轮对话 ===")
response2 = chat.user("能告诉我今天是星期几吗？").get_response()
print(f"用户: 能告诉我今天是星期几吗？")
print(f"助手: {response2.content}")
print(f"Token使用: {response2.total_tokens}")

=== 第二轮对话 ===
用户: 能告诉我今天是星期几吗？
助手: 今天是星期四。
Token使用: 100


## 4. 查看对话历史

In [9]:
# 查看完整对话历史
print(f"\n总消息数: {len(chat)}")
print(f"最后一条消息: {chat.last_message}")
print("=== 对话历史 ===")
chat.print_log()


总消息数: 5
最后一条消息: 今天是星期四。
=== 对话历史 ===

--------------------------------------------------
SYSTEM
--------------------------------------------------
你是一个友善的AI助手，请用简洁明了的方式回答问题。

--------------------------------------------------
USER
--------------------------------------------------
你好！请简单介绍一下自己。

--------------------------------------------------
ASSISTANT
--------------------------------------------------
你好！我是一个由人工智能驱动的助手，能够帮助回答问题、提供信息、协助写作和解决各种任务。如果你有任何问题，随时告诉我！

--------------------------------------------------
USER
--------------------------------------------------
能告诉我今天是星期几吗？

--------------------------------------------------
ASSISTANT
--------------------------------------------------
今天是星期四。


In [10]:
chat.print_debug_info()

=== Chat Debug Info ===
message_count: 5
model: gpt-3.5-turbo
api_base: https://4.0.wokaai.com/v1
last_response: {'id': 'chatcmpl-Cc5kGVPUki8ImDteTVN6rbtcHdAlD', 'model': 'gpt-4.1-mini-2025-04-14', 'created': 1763194584, 'usage': {'completion_tokens': 6, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens': 94, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}, 'total_tokens': 100}, 'finish_reason': 'stop', 'is_valid': True, 'is_stream': False}


## 5. 便捷问答方法

In [11]:
# 使用 ask 方法进行快速问答
print("=== 便捷问答 ===")
chat.clear()
answer = chat.ask("请用一句话解释什么是人工智能？", update_history=False)
print(f"问题: 请用一句话解释什么是人工智能？")
print(f"回答: {answer}")
print(len(chat))

=== 便捷问答 ===
问题: 请用一句话解释什么是人工智能？
回答: 人工智能是指通过计算机系统模拟人类智能，以实现感知、学习、推理和决策等能力的技术。
0


## 6. 异步对话示例

In [4]:
# 异步对话函数
print("=== 异步对话示例 ===")

# 创建新的对话实例
async_chat = Chat(model='gpt-3.5-turbo')
async_chat.system("你是一个编程助手，请提供简洁的编程建议。")

# 异步获取响应
response = await async_chat.async_ask("计算 34 * 33")
print(f"回答: {response}")
print(len(async_chat))

=== 异步对话示例 ===
回答: 34 * 33 = 1122
3


# 7. 流式对话示例

In [7]:
chat = Chat('write an essay of sun', model='gpt-3.5-turbo')

# 使用异步流式响应
accumulated_content = ""

async for response in chat.async_get_response_stream():
    # 获取增量内容
    if response.delta_content:
        accumulated_content += response.delta_content
        print(response.delta_content, end='', flush=True)
print()
print(f"完整内容长度: {len(accumulated_content)}")

**The Sun: The Life-Giving Star**

The sun is the most important celestial body in our solar system. It is a massive, glowing sphere of hot gases that provides the energy necessary for life on Earth. Without the sun, our planet would be a frozen, lifeless rock drifting in the dark void of space. This essay explores the nature of the sun, its structure, its significance to life on Earth, and its influence on human culture.

The sun is classified as a G-type main-sequence star, often called a yellow dwarf. It is composed primarily of hydrogen (about 74%) and helium (about 24%), with trace amounts of heavier elements. The sun’s core is where nuclear fusion occurs—hydrogen atoms fuse to form helium, releasing an enormous amount of energy in the form of light and heat. This energy radiates outward through the sun’s layers and into space, reaching Earth in about 8 minutes and 20 seconds.

Structurally, the sun consists of several layers. At the center lies the core, where the energy is produ